In [1]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

# pip install numpy
# conda install -c anaconda pandas
# conda install -c conda-forge matplotlib

import datetime as dt # For defining dates

import time

# In Powershell Prompt : conda install -c conda-forge multitasking
# pip install -i https://pypi.anaconda.org/ranaroussi/simple yfinance

import yfinance as yf

# To show all your output File -> Preferences -> Settings Search for Notebook
# Notebook Output Text Line Limit and set to 100

# Used for file handling like deleting files
import os

# conda install -c conda-forge cufflinks-py
# conda install -c plotly plotly
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

from plotly.subplots import make_subplots

# New Imports
# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

import warnings
warnings.simplefilter("ignore")

**Constants**

In [2]:
# Save the stocks data downloaded
PATH = '../raw_data/data_stocks/'
# File with all the tickers to be downloaded
tickers_path = '../raw_data/ibove_conso/all_tickers.csv'
# Save stocks data modified
path_folder = '../raw_data/stocks_analysis/'

#Start end date defaults
S_DATE = '2017-02-01'
E_DATE = '2022-12-06'
S_DATE_DT = pd.to_datetime(S_DATE)
E_DATE_DT = pd.to_datetime(E_DATE)

# Stocks Data Retrieve

## Get Dataframe from CSV

In [3]:
# Reads a dataframe from the CSV file, changes index to date and returns it
def get_stock_df_from_csv(ticker):
    
    # Try to get the file and if it doesn't exist issue a warning
    try:
        df = pd.read_csv(PATH + ticker + '.csv', index_col=0)
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df

## Get all Stocks Downloaded in List

In [5]:
files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
tickers
tickers.remove('.DS_Store') # MacOS Only
tickers.sort()
len(tickers)

93

# Finance Calcs: Daily returns, Cumulative Bollinger Bands & Ichimoku to Dataframes

##  Add Daily Returns

In [6]:
# Shift provides the value from the previous day
# NaN is displayed because there was no previous day price for the 1st calculation
def add_daily_return_to_df(df):
    df['daily_return'] = (df['Close'] / df['Close'].shift(1)) - 1
    # Save data to a CSV file
    # df.to_csv(path_folder + ticker + '.csv')
    return df 

## Add Cumulative Returns

In [7]:
def add_cum_return_to_df(df):
    df['cum_return'] = (1 + df['daily_return']).cumprod()
    # df.to_csv(path_folder + ticker + '.csv')
    return df

## Add Bollinger Bands

In [8]:
# Here we will add a middle band (20 days), upper band (20 days + 1.96 std),
# and lower band (20 days - 1.96 std)
def add_bollinger_bands(df):
    df['middle_band'] = df['Close'].rolling(window=20).mean()
    df['upper_band'] = df['middle_band'] + 1.96 * df['Close'].rolling(window=20).std()
    df['lower_band'] = df['middle_band'] - 1.96 * df['Close'].rolling(window=20).std()
    # df.to_csv(path_folder + ticker + '.csv')
    return df

## Add Ichimoku Data to Dataframe

In [9]:
def add_Ichimoku(df):
    # Conversion = Represents support, resistance and reversals.
    #Used to measure short term trends.
    hi_val = df['High'].rolling(window=9).max()
    low_val = df['Low'].rolling(window=9).min()
    df['Conversion'] = (hi_val + low_val) / 2

    # Baseline = (Highest Value in period + Lowest value in period)/2 (26 Sessions)
    hi_val2 = df['High'].rolling(window=26).max()
    low_val2 = df['Low'].rolling(window=26).min()
    df['Baseline'] = (hi_val2 + low_val2) / 2

    # Spans
    # Span A = (Conversion Value + Base Value)/2 (26 Sessions)
    df['SpanA'] = ((df['Conversion'] + df['Baseline']) / 2).shift(26)
    
    # Span B = (Conversion Value + Base Value)/2 (52 Sessions)
    hi_val3 = df['High'].rolling(window=52).max()
    low_val3 = df['Low'].rolling(window=52).min()
    df['SpanB'] = ((hi_val3 + low_val3) / 2).shift(26)
    
    # Lagging Span = Price shifted back 26 periods
    df['Lagging'] = df['Close'].shift(-26)

    return df

## Test functions individually before running for the all the stocks

In [10]:
# It is always better to test your results using one stock
try:
    print("Working on :", "WEGE3,SA")
    new_df = get_stock_df_from_csv("WEGE3,SA")
    new_df = add_daily_return_to_df(new_df)
    new_df = add_cum_return_to_df(new_df)
    new_df = add_bollinger_bands(new_df)
    new_df = add_Ichimoku(new_df)
    new_df.to_csv(path_folder + 'WEGE3,SA' + '.csv')
except Exception as ex:
    print(ex)

Working on : WEGE3,SA


##  Add Daily, Cumulative Bollinger Bands & Ichimoku to Dataframes

In [11]:
for x in tickers:
    try:
        print("Working on :", x)
        new_df = get_stock_df_from_csv(x)
        new_df = add_daily_return_to_df(new_df)
        new_df = add_cum_return_to_df(new_df)
        new_df = add_bollinger_bands(new_df)
        new_df = add_Ichimoku(new_df)
        new_df.to_csv(path_folder + x + '.csv')
    except Exception as ex:
        print(ex)

Working on : .csv
File Doesn't Exist
'NoneType' object is not subscriptable
Working on : ABEV3,SA
Working on : ALPA4,SA
Working on : AMER3,SA
Working on : ASAI3,SA
Working on : AZUL4,SA
Working on : B3SA3,SA
Working on : BBAS3,SA
Working on : BBDC3,SA
Working on : BBDC4,SA
Working on : BBSE3,SA
Working on : BEEF3,SA
Working on : BIDI11,SA
Working on : BPAC11,SA
Working on : BPAN4,SA
Working on : BRAP4,SA
Working on : BRFS3,SA
Working on : BRKM5,SA
Working on : BRML3,SA
Working on : CASH3,SA
Working on : CCRO3,SA
Working on : CIEL3,SA
Working on : CMIG4,SA
Working on : CMIN3,SA
Working on : COGN3,SA
Working on : CPFE3,SA
Working on : CPLE6,SA
Working on : CRFB3,SA
Working on : CSAN3,SA
Working on : CSNA3,SA
Working on : CVCB3,SA
Working on : CYRE3,SA
Working on : DXCO3,SA
Working on : ECOR3,SA
Working on : EGIE3,SA
Working on : ELET3,SA
Working on : ELET6,SA
Working on : EMBR3,SA
Working on : ENBR3,SA
Working on : ENEV3,SA
Working on : ENGI11,SA
Working on : EQTL3,SA
Working on : EZTC3,